## Tensorboard Visualization of the feature vectors

In [2]:
# Imports.
import tensorflow as tf
import keras
import keras.backend as K
from tensorboard.plugins import projector
from scripts.losses import offline_triplet_loss
from tensorflow_addons.losses.triplet import triplet_semihard_loss

import os
import h5py
import numpy as np

from scripts.data import prepare_pose, read_data, augment_data, split_data_metrics_learning, get_req_ids, train_test, subsample
from scripts.evaluate import generate_embedding, build_annoy_index
from scripts.metrics import get_metrics

In [5]:
# Read data.h5 and preprocess the poses.
extracted_poses, transformed_poses, target = read_data(path="./data/data_v2.h5")
person_ids = target[:,0]
seat_ids = target[:,1]
actual_pose = prepare_pose(transformed_poses, seat_ids)

In [4]:
# Set configurations.
cfg = {
    #data config
    'actual_pose': actual_pose,
    'target': target,
    'person_ids': person_ids,
    'val_ids': [1, 3],
    'num_of_val_ids': 5,
    'window_width' : 90,
    'overlap': 0.5,
    'random_state': 32,
    'augment_data': True, # or False
    'anchor_gallery_split_size': 0.2,
    'num_of_joints': 11,
        
    #triplet mining
    'mining': 'offline', # or 'online' 
    
    #model config
    'vec_dim': 128,
    'model_path' : './models/model-val-[1, 3, 4, 19, 20]-90-128-offline.h5',
    
    #tensorboard
    'logdir': "tensorboard_logdir/visual_dir1/",
    'viz_mode': 'both' # or anchor or both
}

In [4]:
# This cell contains splits anchor and gallery set.

X_set, y_set = get_req_ids(cfg['actual_pose'], cfg['target'], cfg['val_ids'], cfg['person_ids'])
X_gal, X_anchor, y_gal, y_anchor = train_test(X_set, y_set, test_size=cfg['anchor_gallery_split_size'], random_state=cfg['random_state'], stratify=y_set)

X_gal, y_gal = subsample(cfg=cfg, poses=X_gal, targets=y_gal, window_width = cfg['window_width'], overlap = cfg['overlap'])
X_anchor, y_anchor = subsample(cfg=cfg, poses=X_gal, targets=y_gal, window_width = cfg['window_width'], overlap = cfg['overlap'])

y_anchor += 100

X_set = np.concatenate((X_gal, X_anchor))
y_set = np.concatenate((y_gal, y_anchor))

In [5]:
 # log directory for storing tensorboard generated files.
LOG_DIR = os.path.join(os.getcwd(), cfg['logdir'])

# Load model from the model_path in the cfg.
model = None
model = keras.models.load_model(cfg['model_path'], custom_objects={'offline_triplet_loss': offline_triplet_loss, 'tf':tf})

# Select the data for visualization. 
# Ex. gallery => visualize the feature vectors of the gallery set.
# Ex. anchor  => visualize the feature vectors of the anchor set.
# Ex. both => visualize the feature vectors of both the gallery set and the anchor set together.

if cfg['viz_mode'] == 'gallery':
    data = X_gal
    label = y_gal
elif cfg['viz_mode'] == 'anchor':
    data = X_anchor
    label = y_anchor
elif cfg['viz_mode'] == 'both':
    data = X_set
    label = y_set
    
# Get all the feature vectors and put it in a dictionary.
embedding_dict = generate_embedding(cfg, model, data)

# Read all the feature vectors from the dictionary and store it in a numpy array.
embeddings_array = np.zeros((len(embedding_dict), 128))
for i, embed in enumerate(embedding_dict.values()):
    embeddings_array[i] = embed
    
print("Embedding array : ", embeddings_array.shape)

In [6]:
 # Load the features in a tensorflow variable 
features = tf.Variable(embeddings_array, name='features')

# Create checkpoint files for the model.
checkpoint = tf.train.Checkpoint(embedding=model)
checkpoint.save(os.path.join(LOG_DIR, "model.ckpt"))

# Create a csv file for the label.
np.savetxt(fname=os.path.join(LOG_DIR, "metadata.csv"), X=label, fmt='%0.0f')

# Create a tf session.
sess = tf.compat.v1.Session()
saver = tf.compat.v1.train.Saver([features])
saver.save(sess, os.path.join(LOG_DIR, 'model.ckpt'))

# Create a embedding projector config and write all the metadata on to it.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = features.name
embedding.metadata_path = os.path.join(LOG_DIR, "metadata.csv")

# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(LOG_DIR, config)